# VDM Corner Testbed - Baseline vs VDM Regularizer (skeleton)
This notebook loads a single YAML config and provides stub functions for:
1) Baseline corner simulation; 2) VDM-regularized run; 3) Figures.

> Replace the stubs with your solver. Keep the config as the single source of truth.

In [ ]:
# Load config and show geometry
import yaml, json, os
import matplotlib.pyplot as plt
from PIL import Image
cfg_path = 'VDM_corner_config.yaml'
assert os.path.exists(cfg_path), 'Put VDM_corner_config.yaml next to this notebook.'
with open(cfg_path) as f:
    CFG = yaml.safe_load(f)
CFG

In [ ]:
# Display geometry figure (already generated)
from IPython.display import Image as DImage, display
if os.path.exists('VDM_Corner_Geometry.png'):
    display(DImage(filename='VDM_Corner_Geometry.png'))
else:
    print('Geometry figure not found; regenerate if needed.')

## Baseline solver stub
**Terse:** compute `u(x,y,t)` for rc, H, U0, ν with your preferred scheme.

**Expanded:** Implement your 2‑D solver (FD/FE/LBM). Enforce inlet Dirichlet, walls no‑slip, outlet Neumann. Save time trace of `max|u|`, and snapshots of streamlines/vorticity at a chosen time.

In [ ]:
# --- REPLACE THIS WITH YOUR SOLVER ---
import numpy as np

def simulate_corner_baseline(cfg):
    # Placeholder: return dummy arrays with the right shapes/keys so plotting cells run.
    t = np.linspace(0, cfg['t_end'], int(cfg['t_end']/cfg['dt'])+1)
    maxspeed = np.minimum(cfg['U0']*(1+0.5*np.sin(4*np.pi*t)), cfg['U0']*5)
    # Placeholders for 2D fields at a single snapshot
    nx, ny = cfg['Nx'], cfg['Ny']
    stream = np.random.rand(ny, nx) * 0.0  # replace with your streamfunction
    vort   = np.random.randn(ny, nx) * 0.0  # replace with vorticity
    return {'t': t, 'maxspeed': maxspeed, 'stream': stream, 'vort': vort}

baseline = simulate_corner_baseline(CFG)
baseline['t'][:5], baseline['maxspeed'][:5]

## VDM regularizer (toy testbed) stub
**Terse:** evolve scalar `D` with `∂t D = α||∇u||² - D/τ_r + κΔD`, scale advection as `(u·∇)u → (1/(1+βD))(u·∇)u`.

**Expanded:** Treat `D` as a local budget. High strain increases `D`; it relaxes/diffuses over time. Use the same numerics as baseline plus an extra scalar PDE for `D`. Validate by ablation: `β=0` should match baseline.

In [ ]:
# --- REPLACE THIS WITH YOUR REGULARIZED SOLVER ---
import numpy as np

def simulate_corner_vdm(cfg):
    beta = cfg['regularizer']['beta'] if cfg['regularizer'].get('enabled', False) else 0.0
    t = np.linspace(0, cfg['t_end'], int(cfg['t_end']/cfg['dt'])+1)
    # Placeholder: show a capped growth compared to baseline to illustrate expected behavior
    maxspeed = np.minimum(cfg['U0']*(1+0.5*np.sin(4*np.pi*t)), cfg['U0']*(3.0/(1+beta)))
    nx, ny = cfg['Nx'], cfg['Ny']
    stream = np.random.rand(ny, nx) * 0.0
    vort   = np.random.randn(ny, nx) * 0.0
    return {'t': t, 'maxspeed': maxspeed, 'stream': stream, 'vort': vort}

vdm = simulate_corner_vdm(CFG)
vdm['t'][:5], vdm['maxspeed'][:5]

## Plots

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(6,3), dpi=150)
plt.plot(baseline['t'], baseline['maxspeed'], label='baseline')
plt.plot(vdm['t'], vdm['maxspeed'], label='VDM (β={})'.format(CFG['regularizer']['beta']))
plt.xlabel('time'); plt.ylabel('max |u|'); plt.legend(); plt.title('Max speed vs time')
plt.tight_layout(); plt.show()

### Snapshot placeholders (replace once your solver returns real fields)

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(8,3), dpi=150)
axs[0].imshow(baseline['vort'], origin='lower', cmap='coolwarm')
axs[0].set_title('Baseline vorticity (placeholder)')
axs[1].imshow(vdm['vort'], origin='lower', cmap='coolwarm')
axs[1].set_title('VDM vorticity (placeholder)')
for ax in axs: ax.set_xticks([]); ax.set_yticks([])
plt.tight_layout(); plt.show()

> **Next steps:**
- Replace stubs with your real solver.
- Keep `CFG` as the single source of truth.
- For ablations: set `regularizer.enabled=false` or `beta=0`.
- For “geometry regularization”: sweep `rc` in the YAML.